# 特征：
- 使用Distance（空值填1）
- Discount_rate做one-hot
- 每个用户：优惠券使用次数
- 领券是星期几
- 是否小于15天消费
- 商家优惠券被消费次数
- 用户当天使用优惠券
- 只在一个商铺领取过优惠劵并且有消费记录
- 用户使用该商家的优惠劵的平均时间
- 用户消费的平均距离
- 用户使用某种类型的优惠劵数目/用户领取优惠劵总数
- 用户领取某种类型的优惠劵数目/用户领取优惠劵总数
- 用户在某个商铺的优惠券领取数
- 用户在某个商铺的优惠券消费数
- 用户只领取过一次优惠劵但未消费
- 用户领取优惠劵并且全部消费
- 用户在该商家购物次数超过1次
- 用户领取某商家的优惠劵总数/用户领取的优惠劵总数
- 用户使用某商家的优惠劵总数/用户总体使用优惠劵总数
- 用户在某个商铺领取过优惠券User_Merchant_havereceived_Coupon
- 用户领取过优惠券有多少没有消费Coupon_not_Consume
- 用户在某个商铺领取过优惠券，并且全部消费User_Merchant_all_Consume
- 用户在某个商铺领取过优惠券多少没有消费User_Merchant_not_Consume
- 用户领取某种类型的优惠券，并且全部消费User_Discount_all_Consume
- 用户领取某种类型的优惠券，多少没有消费User_Discount_not_Consume
- 用户领取某个距离的优惠券，并且全部消费User_Distance_all_Consume
- 用户领取某个距离的优惠券，多少没有消费User_Distance_not_Consume
- 用户领取满减券是满多少discount_up
- 用户领取满减券是减多少discount_bottom
# 算法：逻辑回归

In [1]:
import pandas as pd
import numpy as np

off_train = pd.read_csv('E:/tianchi/ccf_offline_stage1_train.csv')
off_test = pd.read_csv('E:/tianchi/ccf_offline_stage1_test_revised.csv')
off_train = off_train.dropna(subset = ['Coupon_id'])

# 加标签

In [2]:
off_train['result'] = off_train[['Coupon_id', 'Date']].apply(lambda x: 1 if pd.notnull(x[0]) and pd.notnull(x[1]) else 0, axis = 1)

# 加领券是星期几

In [3]:
#对训练集取星期数
off_train["Week"] = off_train["Date_received"]
off_train.Week = off_train.Week.astype("int")
off_train.Week = off_train.Week.astype("str")
off_train["Week"] = pd.to_datetime(off_train["Week"]).apply(lambda x : x.weekday())

#对测试集取星期数
off_test["Week"] = off_test["Date_received"]
off_test.Week = off_test.Week.astype("int")
off_test.Week = off_test.Week.astype("str")
off_test["Week"] = pd.to_datetime(off_test["Week"]).apply(lambda x : x.weekday())

# 加是否小于15天消费

In [4]:
import datetime

#判断使用优惠券日期-领取优惠券日期是否小于15天，小于取1，大于取0 
def days(z):
    if str(z['Date_received']) != 'nan' and str(z['Date']) != 'nan':
        days = (datetime.datetime.strptime(str(int(z['Date'])), "%Y%m%d") - datetime.datetime.strptime(str(int(z['Date_received'])), "%Y%m%d"))
        if days.days > 15:
            return 0
        else:
            return 1
    else:
        return 0

#将判断后的结果赋值给训练集    
off_train['less_15'] = off_train.apply(days, axis = 1)

#用户和优惠券id是多对多的关系，所以按用户对他消费小于15天的数做了一个累加，去除掉训练集中的这列特征，方便做处理后特征的连接
train = off_train[['User_id', 'less_15']]
train = train.drop_duplicates(subset = ['User_id'])
a = train.less_15.groupby(train['User_id']).sum()
train = a.reset_index()
off_train = off_train.drop(columns = 'less_15')

#对测试集和测试集按用户增加这列特征
off_train = pd.merge(off_train, train, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, train, on = 'User_id', how = 'left')

#测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_test.less_15 = off_test.less_15.fillna(0)

# 加用户消费次数

In [5]:
#统计用户消费次数
User_Coupon_notna = off_train[['User_id', 'Date']].groupby('User_id').count()
User_Coupon_notna = User_Coupon_notna.reset_index()
User_Coupon_notna.rename(columns = {'Coupon_id' : 'Coupon_id_count', 'Date' : 'Consume_count'}, inplace=True)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, User_Coupon_notna[['User_id', 'Consume_count']], on = ['User_id'], how = 'left')
off_test = pd.merge(off_test, User_Coupon_notna[['User_id', 'Consume_count']], on = 'User_id', how = 'left')
off_test = off_test.fillna(value = {'Consume_count' : 0})

# Distinct填1

In [6]:
off_train.Distance = off_train.Distance.fillna(1)
off_test.Distance = off_test.Distance.fillna(1)

# 加商家优惠券被消费次数

In [7]:
#统计商家优惠券消费次数
Merchant_Coupon_notna = off_train['Date'].groupby(off_train['Merchant_id']).count()
Merchant_Coupon_notna = Merchant_Coupon_notna.reset_index()
Merchant_Coupon_notna.rename(columns = {'Date' : 'Coupon_have_userd_counts'}, inplace = True)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的商家，这类商家的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, Merchant_Coupon_notna, on = ['Merchant_id'], how = 'left')
off_test = pd.merge(off_test, Merchant_Coupon_notna ,on = ['Merchant_id'], how='left')
off_test = off_test.fillna(value = {'Coupon_have_userd_counts' : 0})

# 加用户当天使用优惠券

In [8]:
#找出有优惠券消费的数据
date_nan_drop = off_train.dropna(subset=['Date'])
date_nan_drop = date_nan_drop.reset_index()

#领取日期等于消费日期，说明是当天消费
import datetime
def Use(z):
    if z.Date_received == z.Date:
        return 1
    else:
        return np.NaN
date_nan_drop['Use_today'] = date_nan_drop.apply(Use, axis=1)
#一个用户可能有多个当天消费行为，将这部分去重
a = date_nan_drop[['User_id', 'Use_today']]
a = a.drop_duplicates(subset=['User_id'])
#将结果返回
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')
#空值填0
off_train = off_train.fillna({'Use_today':0})
off_test = off_test.fillna({'Use_today':0})

# 加只在一个商铺领取过优惠劵并且有消费记录

In [9]:
#找出有优惠券消费的数据
date_nan_drop = off_train.dropna(subset=['Date'])
date_nan_drop = date_nan_drop.reset_index()
#找出用户在几个商家消费过，只在1个置1，其余置0
Consume_oneMerchant = date_nan_drop['Merchant_id'].groupby(date_nan_drop['User_id'])
a = Consume_oneMerchant.count()
a = a.reset_index()
a = a.rename(columns={'Merchant_id':'Use_One_Merchant'})
a['Use_One_Merchant'] = a.Use_One_Merchant.apply(lambda x:1 if x == 1 else 0)
#将结果返回
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')
#空值填0
off_train = off_train.fillna({'Use_One_Merchant':0})
off_test = off_test.fillna({'Use_One_Merchant':0})

# 加用户使用该商家的优惠劵的平均时间

In [10]:
import datetime

#返回领取到消费天数
def days(z):
    if str(z['Date_received']) != 'nan' and str(z['Date']) != 'nan':
        days = (datetime.datetime.strptime(str(int(z['Date'])), "%Y%m%d") - datetime.datetime.strptime(str(int(z['Date_received'])), "%Y%m%d"))
        return days.days
    else:
        return np.NaN
    
off_train['days'] =  off_train.apply(days, axis=1)
#找出有消费的数据
use_Coupon = off_train.dropna(subset=['Date'])
#按用户计算天数均值
a = use_Coupon['days'].groupby(use_Coupon['User_id']).mean()
a = a.reset_index()
a = a.rename(columns={'days':'Use_mean_time'})
#将结果返回测试集与训练集
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')
#空值填0
off_train = off_train.fillna({'Use_mean_time':0})
off_test = off_test.fillna({'Use_mean_time':0})

# 加用户消费的平均距离

In [11]:
#找出有消费的数据
use_Coupon = off_train.dropna(subset=['Date'])
#按用户计算距离均值
a = use_Coupon['Distance'].groupby(use_Coupon['User_id']).mean()
a = a.reset_index()
a = a.rename(columns={'Distance':'Use_mean_Distance'})
#将结果返回训练集与测试集
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')
#空值填0
off_train = off_train.fillna({'Use_mean_Distance':0})
off_test = off_test.fillna({'Use_mean_Distance':0})

# 加用户使用某种类型的优惠劵数目/用户领取优惠劵总数

In [12]:
#找出有消费的数据
use_Coupon = off_train.dropna(subset=['Date'])
#找出用户使用某种类型的优惠券数目
a = use_Coupon[['User_id', 'Discount_rate']].groupby([use_Coupon['User_id'], use_Coupon['Discount_rate']]).count()
a = a.rename(columns={'User_id':'Coupon_use'})
a = a.drop(columns=['Discount_rate'])
a = a.reset_index()
#找不用户领取优惠券总数
b = off_train['Date_received'].groupby(off_train['User_id']).count()
b = b.reset_index()
b = b.rename(columns={'Date_received':'Coupon_recieved'})
#将他们相除
c = pd.merge(a, b, on = 'User_id', how = 'left')
def chufa(z):
    x = z.Coupon_use / z.Coupon_recieved
    return x
c['Coupon_use/Coupon_sum'] = c.apply(chufa, axis=1)
#将结果返回训练集与测试集
off_train = pd.merge(off_train, c[['User_id', 'Discount_rate', 'Coupon_use/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
off_test = pd.merge(off_test, c[['User_id', 'Discount_rate', 'Coupon_use/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
#空值填0
off_train = off_train.fillna({'Coupon_use/Coupon_sum':0})
off_test = off_test.fillna({'Coupon_use/Coupon_sum':0})

# 加用户领取某种类型的优惠劵数目/用户领取优惠劵总数

In [13]:
#找出用户领取某种类型的优惠劵数目
a = off_train[['User_id', 'Discount_rate', 'Date_received']].groupby(['User_id', 'Discount_rate']).count()
a = a.rename(columns={'Date_received':'Coupon_use'})
a = a.reset_index()
#找出用户领取优惠劵总数
b = off_train['Date_received'].groupby(off_train['User_id']).count()
b = b.reset_index()
b = b.rename(columns={'Date_received':'Coupon_recieved'})
#将他们相除
c = pd.merge(a, b, on = 'User_id', how = 'left')
def chufa(z):
    x = z.Coupon_use / z.Coupon_recieved
    return x
c['Coupon/Coupon_sum'] = c.apply(chufa, axis=1)
#将结果返回训练集与测试集
off_train = pd.merge(off_train, c[['User_id', 'Discount_rate', 'Coupon/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
off_test = pd.merge(off_test, c[['User_id', 'Discount_rate', 'Coupon/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
#空值填0
off_train = off_train.fillna({'Coupon/Coupon_sum':0})
off_test = off_test.fillna({'Coupon/Coupon_sum':0})

# 加用户在某个商铺的优惠券领取数

In [14]:
#找出用户在某个商铺的优惠券领取数
a = off_train['Date_received'].groupby([off_train['User_id'], off_train['Merchant_id']]).count()
a = a.reset_index()
a = a.rename(columns={'Date_received':'User_Merchant_Coupon'})
#将结果返回训练集与测试集
off_train = pd.merge(off_train, a, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, a, on = ['User_id', 'Merchant_id'], how = 'left')
#空值填0
off_train = off_train.fillna({'User_Merchant_Coupon':0})
off_test = off_test.fillna({'User_Merchant_Coupon':0})

# 加用户在某个商铺的优惠券消费数

In [15]:
#找出用户在某个商铺的优惠券消费数
a = off_train['Date'].groupby([off_train['User_id'], off_train['Merchant_id']]).count()
a = a.reset_index()
a = a.rename(columns={'Date':'User_Merchant_Coupon_used'})
#将结果返回训练集与测试集
off_train = pd.merge(off_train, a, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, a, on = ['User_id', 'Merchant_id'], how = 'left')
#空值填0
off_train = off_train.fillna({'User_Merchant_Coupon_used':0})
off_test = off_test.fillna({'User_Merchant_Coupon_used':0})

# 用户只领取过一次优惠劵但未消费

In [16]:
user_coupon = off_train[['User_id', 'Coupon_id']]
# 利用User_id和Coupon_id两列对用户进性分组，并且计算每组的数目，并进行序号重置
user_coupon_count = user_coupon.groupby(by=['User_id']).size().reset_index()
# 重命名
user_coupon_count.rename(columns={0:'Coupon_size'}, inplace=True)
# 将筛选出来需要merge的列和原始的数据进行merge Coupon_size代表用户领取的优惠劵数目
off_train = off_train.merge(user_coupon_count, on='User_id', how='inner')

user_coupon = off_train[['User_id', 'result']]
# 利用User_id和result两列对用户进性分组，并且计算每组的数目（这里相当于求和），并进行序号重置
user_coupon_used_count = user_coupon.groupby(by=['User_id']).sum().reset_index()
# 重命名 Consume_count用户使用了的优惠劵数目
user_coupon_used_count.rename(columns={'result':'Conupon_have_used_count'}, inplace=True)
# 将筛选出来需要merge的列和原始的数据进行merge
off_train = off_train.merge(user_coupon_used_count, on='User_id', how='inner')

# 为训练集添加Coupon_all_consume特征，标记那些领取优惠劵数目等于使用优惠劵数目
off_train['Coupon_all_consume'] = off_train[['Conupon_have_used_count','Coupon_size']].apply(lambda x : 1 if x[0] == x[1] else 0, axis = 1)
# 去掉产生的中间列
off_train = off_train.drop(columns=['Coupon_size', 'Conupon_have_used_count'])
# 提取需要的列好merge到测试集中
off_train_coupon_all_consume = off_train[['User_id', 'Coupon_all_consume']]
off_train_coupon_all_consume = off_train_coupon_all_consume.drop_duplicates(['User_id', 'Coupon_all_consume'])
#为测试集添加Coupon_all_consume特征
off_test = off_test.merge(off_train_coupon_all_consume, on='User_id', how='left')
# 填充merge以后的空值为0
off_test = off_test.fillna({'Coupon_all_consume':0})

# 用户领取优惠劵并且全部消费

In [17]:
user_coupon = off_train[['User_id', 'Coupon_id','result']]
# 利用User_id和result两列对用户进性分组，并且计算每组的数目，并进行序号重置
user_coupon_count = user_coupon.groupby(by=['User_id','result']).size().reset_index()
# 重命名
user_coupon_count.rename(columns={0:'Coupon_size'}, inplace=True)
# 筛选出领取次数为1并且未消费的
user_coupon_count_1 = user_coupon_count[(user_coupon_count.Coupon_size == 1) & (user_coupon_count.result == 0)]
# 去掉不需要merge的这一列，即result
user_coupon_count_1 = user_coupon_count_1.drop(columns=['result'])

# 训练集进行添加One_coupon_no_consume特征
# 将筛选出来需要merge的列和原始的数据进行merge
off_train = off_train.merge(user_coupon_count_1, on='User_id', how='left')
#再次重命名
off_train.rename(columns={'Coupon_size':'One_coupon_no_consume'}, inplace=True)
# 填充merge以后的空值项为 0
off_train = off_train.fillna({'One_coupon_no_consume':0})

# 测试集添加One_coupon_no_consume特征
off_test = off_test.merge(user_coupon_count_1, on='User_id', how='left')
off_test.rename(columns={'Coupon_size':'One_coupon_no_consume'}, inplace=True)
off_test = off_test.fillna({'One_coupon_no_consume':0})

# 用户在该商家购物次数超过1次

In [18]:
User_Merchant_notna = off_train[['User_id','Merchant_id','Date']].groupby(['User_id','Merchant_id']).count()
User_Merchant_notna = User_Merchant_notna.reset_index()
User_Merchant_notna.rename(columns = {'Date' : 'Consume_count_M'}, inplace=True)  #Consume_count_M表示用户在M商家的使用优惠券的消费次数
print(User_Merchant_notna)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, User_Merchant_notna[['User_id', 'Merchant_id','Consume_count_M']], on = ['User_id','Merchant_id'], how = 'left')
off_test = pd.merge(off_test, User_Merchant_notna[['User_id', 'Merchant_id','Consume_count_M']], on = ['User_id','Merchant_id'], how = 'left')
off_test = off_test.fillna(value = {'Consume_count_M' : 0})
#print (off_train)

#对Consume_count_M进行判别是否大于1，若大于1则添加标签Consume_count_M_over1为1，否则为0
  ##训练集中
def judge_train(off_train) :
    if (off_train['Consume_count_M']>0):
      Consume_count_M_over1=1
    else:
      Consume_count_M_over1=0
    return Consume_count_M_over1
off_train['Consume_count_M_over1']=off_train.apply(judge_train,axis=1)
print (off_train)
  ##测试集中
def judge_test(off_test) :
    if (off_test['Consume_count_M']>0):
      Consume_count_M_over1=1
    else:
      Consume_count_M_over1=0
    return Consume_count_M_over1
off_test['Consume_count_M_over1']=off_test.apply(judge_test,axis=1)

        User_id  Merchant_id  Consume_count_M
0             4         1433                0
1             4         1469                0
2            35         3381                0
3            36         1041                0
4            36         5717                0
5            64         2146                0
6           110         4433                0
7           110         6454                0
8           110         7019                0
9           144         1553                0
10          147         3381                0
11          165         2934                0
12          165         3381                0
13          165         4195                0
14          166          484                0
15          173         3381                0
16          184         3381                1
17          215          129                0
18          228         7019                0
19          236         1041                0
20          236         1433      

# 用户领取某商家的优惠劵总数/用户领取的优惠劵总数

In [19]:
#按用户和商家id对用户，优惠券id和商家id进行分组并计数，得到用户领取某商家的优惠券次数/根据用户对优惠券id进行分组得到的用户领取优惠券的总数
User_Merchant_Coupon_notna=off_train[['User_id','Merchant_id','Date_received']].groupby(['User_id','Merchant_id']).count()/off_train[['User_id','Date_received']].groupby('User_id').count()
User_Merchant_Coupon_notna = User_Merchant_Coupon_notna.reset_index()
User_Merchant_Coupon_notna.rename(columns = {'Date_received' : 'User_Merchant_Coupon_percentage'}, inplace=True)  #User_Merchant_Coupon_percentage表示用户领取某商家的优惠劵总数/用户领取的优惠劵总数
print(User_Merchant_Coupon_notna)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, User_Merchant_Coupon_notna[['User_id', 'Merchant_id','User_Merchant_Coupon_percentage']], on = ['User_id','Merchant_id'], how = 'left')
off_test = pd.merge(off_test, User_Merchant_Coupon_notna[['User_id', 'Merchant_id','User_Merchant_Coupon_percentage']], on = ['User_id','Merchant_id'], how = 'left')
off_test = off_test.fillna(value = {'User_Merchant_Coupon_percentage' : 0})
#print (off_train)

        User_id  Merchant_id  User_Merchant_Coupon_percentage
0             4         1433                         0.500000
1             4         1469                         0.500000
2            35         3381                         1.000000
3            36         1041                         0.500000
4            36         5717                         0.500000
5            64         2146                         1.000000
6           110         4433                         0.333333
7           110         6454                         0.333333
8           110         7019                         0.333333
9           144         1553                         1.000000
10          147         3381                         1.000000
11          165         2934                         0.333333
12          165         3381                         0.333333
13          165         4195                         0.333333
14          166          484                         1.000000
15      

# 用户使用某商家的优惠劵总数/用户总体使用优惠劵总数

In [20]:
User_Merchant_Coupon_Date_notna = off_train[['User_id','Merchant_id','Date']].groupby(['User_id','Merchant_id']).count()/off_train[['User_id','Date']].groupby(['User_id']).count()
User_Merchant_Coupon_Date_notna = User_Merchant_Coupon_Date_notna.reset_index()
User_Merchant_Coupon_Date_notna.rename(columns = {'Date' : 'User_Merchant_Coupon_Date_percentage'}, inplace=True)  #User_Merchant_Coupon_Date_percentage表示用户使用某商家的优惠劵总数/用户总体使用优惠劵总数
User_Merchant_Coupon_Date_notna=User_Merchant_Coupon_Date_notna.fillna(value={'User_Merchant_Coupon_Date_percentage':0})
print(User_Merchant_Coupon_Date_notna)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, User_Merchant_Coupon_Date_notna[['User_id', 'Merchant_id','User_Merchant_Coupon_Date_percentage']], on = ['User_id','Merchant_id'], how = 'left')
off_test = pd.merge(off_test, User_Merchant_Coupon_Date_notna[['User_id', 'Merchant_id','User_Merchant_Coupon_Date_percentage']], on = ['User_id','Merchant_id'], how = 'left')
off_test = off_test.fillna(value = {'User_Merchant_Coupon_Date_percentage' : 0})
#print (off_train)

        User_id  Merchant_id  User_Merchant_Coupon_Date_percentage
0             4         1433                                   0.0
1             4         1469                                   0.0
2            35         3381                                   0.0
3            36         1041                                   0.0
4            36         5717                                   0.0
5            64         2146                                   0.0
6           110         4433                                   0.0
7           110         6454                                   0.0
8           110         7019                                   0.0
9           144         1553                                   0.0
10          147         3381                                   0.0
11          165         2934                                   0.0
12          165         3381                                   0.0
13          165         4195                                  

# 用户在某个商铺领取过优惠券User_Merchant_havereceived_Coupon

In [21]:
#找出用户在哪些商铺领取过优惠券，置1
a = off_train[['User_id', 'Merchant_id', 'Date_received']].groupby([off_train['User_id'], off_train['Merchant_id']], as_index=False).count()
a = a.rename(columns={'Date_received':'User_Merchant_havereceived_Coupon'})
a['User_Merchant_havereceived_Coupon'] = a['User_Merchant_havereceived_Coupon'].apply(lambda x: 1)
#将结果返回
off_train = pd.merge(off_train, a, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, a, on = ['User_id', 'Merchant_id'], how = 'left')
#填充空值为0
off_train = off_train.fillna({'User_Merchant_havereceived_Coupon':0})
off_test = off_test.fillna({'User_Merchant_havereceived_Coupon':0})

# 用户领取过优惠券有多少没有消费Coupon_not_Consume

In [22]:
User_Coupon_notna = off_train[['User_id', 'Date']].groupby('User_id', as_index=False).count()
User_Coupon_notna.rename(columns = {'Date' : 'Consume_count'}, inplace=True)

User_Coupon = off_train[['User_id', 'Date_received']].groupby('User_id', as_index=False).count()
User_Coupon.rename(columns = {'Date_received' : 'Coupon_count'}, inplace=True)

User_Coupon['Coupon_not_Consume'] = User_Coupon.Coupon_count - User_Coupon_notna.Consume_count
User_Coupon = User_Coupon.drop(columns=['Coupon_count'])

off_train = pd.merge(off_train, User_Coupon, on = ['User_id'], how = 'left')
off_test = pd.merge(off_test, User_Coupon, on = ['User_id'], how = 'left')

off_test = off_test.fillna({'Coupon_not_Consume':-1})

# 用户在某个商铺领取过优惠券，并且全部消费User_Merchant_all_Consume

In [23]:
a = off_train[['User_id', 'Merchant_id', 'Date_received']].groupby(['User_id', 'Merchant_id'], as_index=False).count()
a = a.rename(columns={'Date_received':'User_Merchant_Coupon'})

b = off_train[['User_id', 'Merchant_id', 'Date']].groupby(['User_id', 'Merchant_id'], as_index=False).count()
b = b.rename(columns={'Date':'User_Merchant_Coupon'})

b['User_Merchant_all_Consume'] = a.User_Merchant_Coupon - b.User_Merchant_Coupon
b['User_Merchant_all_Consume'] = b.User_Merchant_all_Consume.apply(lambda x: 1 if x == 0 else 0)
b = b.drop(columns=['User_Merchant_Coupon'])

off_train = pd.merge(off_train, b, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, b, on = ['User_id', 'Merchant_id'], how = 'left')
#填充空值为0
off_test = off_test.fillna({'User_Merchant_all_Consume':0})

# 用户在某个商铺领取过优惠券多少没有消费User_Merchant_not_Consume

In [24]:
a = off_train[['User_id', 'Merchant_id', 'Date_received']].groupby(['User_id', 'Merchant_id'], as_index=False).count()
a = a.rename(columns={'Date_received':'User_Merchant_Coupon'})

b = off_train[['User_id', 'Merchant_id', 'Date']].groupby(['User_id', 'Merchant_id'], as_index=False).count()
b = b.rename(columns={'Date':'User_Merchant_Coupon'})

b['User_Merchant_not_Consume'] = a.User_Merchant_Coupon - b.User_Merchant_Coupon
b = b.drop(columns=['User_Merchant_Coupon'])

off_train = pd.merge(off_train, b, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, b, on = ['User_id', 'Merchant_id'], how = 'left')

off_test = off_test.fillna({'User_Merchant_not_Consume':-1})

# 用户领取某种类型的优惠券，并且全部消费User_Discount_all_Consume

In [25]:
a = off_train[['User_id', 'Discount_rate', 'Date_received']].groupby(['User_id', 'Discount_rate'], as_index=False).count()
a = a.rename(columns={'Date_received':'Coupon_received'})

b= off_train[['User_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Discount_rate'], as_index=False).count()
b = b.rename(columns={'Date':'Coupon_consumed'})

b['User_Discount_all_Consume'] = a.Coupon_received - b.Coupon_consumed
b['User_Discount_all_Consume'] = b.User_Discount_all_Consume.apply(lambda x: 1 if x == 0 else 0)
b = b.drop(columns=['Coupon_consumed'])

off_train = pd.merge(off_train, b, on = ['User_id', 'Discount_rate'], how = 'left')
off_test = pd.merge(off_test, b, on = ['User_id', 'Discount_rate'], how = 'left')
#填充空值为0
off_test = off_test.fillna({'User_Discount_all_Consume':0})

# 用户领取某种类型的优惠券，多少没有消费User_Discount_not_Consume

In [26]:
a = off_train[['User_id', 'Discount_rate', 'Date_received']].groupby(['User_id', 'Discount_rate'], as_index=False).count()
a = a.rename(columns={'Date_received':'Coupon_received'})

b= off_train[['User_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Discount_rate'], as_index=False).count()
b = b.rename(columns={'Date':'Coupon_consumed'})

b['User_Discount_not_Consume'] = a.Coupon_received - b.Coupon_consumed
b = b.drop(columns=['Coupon_consumed'])

off_train = pd.merge(off_train, b, on = ['User_id', 'Discount_rate'], how = 'left')
off_test = pd.merge(off_test, b, on = ['User_id', 'Discount_rate'], how = 'left')
#填充空值为0
off_test = off_test.fillna({'User_Discount_not_Consume':0})

# 用户领取某个距离的优惠券，并且全部消费User_Distance_all_Consume

In [27]:
a = off_train[['User_id', 'Distance', 'Date_received']].groupby(['User_id', 'Distance'], as_index=False).count()
a = a.rename(columns={'Date_received':'Coupon_received'})

b= off_train[['User_id', 'Distance', 'Date']].groupby(['User_id', 'Distance'], as_index=False).count()
b = b.rename(columns={'Date':'Coupon_consumed'})

b['User_Distance_all_Consume'] = a.Coupon_received - b.Coupon_consumed
b['User_Distance_all_Consume'] = b.User_Distance_all_Consume.apply(lambda x: 1 if x == 0 else 0)
b = b.drop(columns=['Coupon_consumed'])

off_train = pd.merge(off_train, b, on = ['User_id', 'Distance'], how = 'left')
off_test = pd.merge(off_test, b, on = ['User_id', 'Distance'], how = 'left')
#填充空值为0
off_test = off_test.fillna({'User_Distance_all_Consume':0})

# 用户领取某个距离的优惠券，多少没有消费User_Distance_not_Consume

In [28]:
a = off_train[['User_id', 'Distance', 'Date_received']].groupby(['User_id', 'Distance'], as_index=False).count()
a = a.rename(columns={'Date_received':'Coupon_received'})

b= off_train[['User_id', 'Distance', 'Date']].groupby(['User_id', 'Distance'], as_index=False).count()
b = b.rename(columns={'Date':'Coupon_consumed'})

b['User_Distance_not_Consume'] = a.Coupon_received - b.Coupon_consumed
b = b.drop(columns=['Coupon_consumed'])

off_train = pd.merge(off_train, b, on = ['User_id', 'Distance'], how = 'left')
off_test = pd.merge(off_test, b, on = ['User_id', 'Distance'], how = 'left')
#填充空值为0
off_test = off_test.fillna({'User_Distance_not_Consume':0})

# 用户领取满减券是满多少discount_up

In [29]:
import re
regex = re.compile('^\d+:\d+$')

def discount(z):
    if re.match(regex, str(z['Discount_rate'])):
        num_max, num_min = z['Discount_rate'].split(':')
        return num_max
    else:
        return np.NaN

off_train['discount_up'] = off_train.apply(discount, axis = 1)
off_test['discount_up'] = off_test.apply(discount, axis = 1)

off_train = off_train.fillna({'discount_up':0})
off_test = off_test.fillna({'discount_up':0})

# 用户领取满减券是减多少discount_bottom

In [30]:
import re
regex = re.compile('^\d+:\d+$')

def discount(z):
    if re.match(regex, str(z['Discount_rate'])):
        num_max, num_min = z['Discount_rate'].split(':')
        return num_min
    else:
        return np.NaN

off_train['discount_bottom'] = off_train.apply(discount, axis = 1)
off_test['discount_bottom'] = off_test.apply(discount, axis = 1)

off_train = off_train.fillna({'discount_bottom':0})
off_test = off_test.fillna({'discount_bottom':0})

# Discount_rate独热

In [31]:
#对训练集和测试集中的Discount_rate进行独热编码
off_train = pd.concat([off_train, pd.get_dummies(off_train.Discount_rate, prefix='Discount_rate')], axis=1)
off_test = pd.concat([off_test, pd.get_dummies(off_test.Discount_rate, prefix='Discount_rate')], axis=1)

#训练集中出现了4个测试集中没有出现的码段，进行填充
m = list(set(off_train.columns.tolist()) - set(off_test.columns.tolist()))
for i in m:
    off_test[i] = 0

#测试集中有一个训练集中没有的码段，进行填充    
off_train['Discount_rate_500:30'] = 0

In [33]:
#训练数据、训练标签、测试数据的设置
x_train = off_train.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Date', 'result', 'days', 'Discount_rate'])
y_train = off_train.result
x_test = off_test.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate'])

#将训练数据和测试数据的列一一对应
x_test = x_test[x_train.columns.tolist()]

from sklearn.linear_model import LogisticRegression

#调用逻辑回归模型，并进行训练
clf = LogisticRegression()
clf.fit(x_train, y_train)

#进行预测，并将需要保存的部分写入save
predict = clf.predict_proba(x_test)
save = off_test[['User_id', 'Coupon_id', 'Date_received']]
save.insert(3, 'probability', predict[:,1], True)

#输出结果为csv
save.to_csv('E:/tianchi/20180728/29_feature.csv', index = None)

In [34]:
save

,User_id,Coupon_id,Date_received,probability
0,4129537,9983,20160712,0.000523
1,6949378,3429,20160706,0.906098
2,2166529,6928,20160727,0.000632
3,2166529,1808,20160727,0.001228
4,6172162,6500,20160708,0.001873
5,4005121,9983,20160706,0.000701
6,4347394,9983,20160716,0.002755
7,3094273,13602,20160727,0.002544
8,5139970,9983,20160729,0.001187
9,3237121,13602,20160703,0.001132


In [35]:
#训练数据、训练标签、测试数据的设置
x_train = off_train.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Date', 'result', 'days', 'Discount_rate'])
y_train = off_train.result
x_test = off_test.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate'])

#将训练数据和测试数据的列一一对应
x_test = x_test[x_train.columns.tolist()]

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

lr = LogisticRegression()
nb = GaussianNB()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
eclf1 = VotingClassifier(estimators=[('lr', lr), ('nb', nb), ('dt', dt), ('rf', rf)], voting='soft')
eclf1.fit(x_train, y_train)

predict = eclf1.predict_proba(x_test)
save = off_test[['User_id', 'Coupon_id', 'Date_received']]
save.insert(3, 'probability', predict[:,1], True)
save.to_csv('E:/tianchi/20180728/29_Classifier.csv', index = None)

In [36]:
save

,User_id,Coupon_id,Date_received,probability
0,4129537,9983,20160712,0.000131
1,6949378,3429,20160706,0.551525
2,2166529,6928,20160727,0.050158
3,2166529,1808,20160727,0.050307
4,6172162,6500,20160708,0.050468
5,4005121,9983,20160706,0.000175
6,4347394,9983,20160716,0.025689
7,3094273,13602,20160727,0.025636
8,5139970,9983,20160729,0.000297
9,3237121,13602,20160703,0.000283
